## Access catalogues data

In [30]:
import pandas as pd
import urllib.parse

# Spreadsheet IDs
spreadsheet_id = '1e7LXTiTli6ChG0NXl1laAfgh2Rl9qwLaContEkeD2tg'
encoded_sheet_name_cat = urllib.parse.quote('Zeri CATALOGHI')
encoded_sheet_name_asta = urllib.parse.quote('Zeri EVENTO ASTA')

# CSV
url_cat = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&sheet={encoded_sheet_name_cat}'
url_asta = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&sheet={encoded_sheet_name_asta}'

# DF
df = pd.read_csv(url_cat)
df_asta = pd.read_csv(url_asta)

display(df_asta.head())

,ID_CATALOGO,INVENTARIO,collocazione,fondi,TITOLO,VENDITA ALL'ASTA,AUTORI_PRINCIPALI/ENTE AUTORE,BANDITORE,AUTORI_SECONDARI/ENTE AUTORE SECONDARIO,LUOGODIVENDITA,DATA INIZIO ASTA,DATA FINE ASTA,CODICEASTA,CRONOLOGIA_OGG_VENDUTI,TIPI_OGGETTI_VENDUTI,COLLEZIONISTI_CONCATENATI
0,14278,87358,CA 18 1882 1110,Germania,Catalog der werthvollen und reichhaltigen samm...,"Asta H.G. Gutekunst, 10-11-1882",Heinrich Gottlob Gutekunst,NaN,NaN,Stoccarda,18821110,nd,29.0,Sec. XV/ XVI,OGGETTI D'ARTE,NaN
1,14320,87399,CA 18 1889 0527,Germania,Katalog der Gemälde-Galerie des verstorbenen H...,"Asta Heberle H. Lempertz' Söhne, 27-05-1889",Heberle H. Lempertz' Söhne <Colonia>,NaN,NaN,Colonia,18890527,18890528,NaN,Sec. XV/ XVIII,DIPINTI,COLLEZIONE FEDOR ZSCHILLE
2,14323,87400,CA 18 1889 0528,Germania,Katalog der Gemälde Sammlung des hern Carl Pag...,"Asta Heberle H. Lempertz' Söhne, 28-05-1889",Heberle H. Lempertz' Söhne <Colonia>,NaN,NaN,Colonia,18890528,18890529,NaN,Sec. XV/ XVIII,DIPINTI,COLLEZIONE CARL PAGENSTECHER
3,14348,87438,CA 18 1892 1114,Germania,Katalog der reichhaltigen und ausgewählten Gem...,"Asta Heberle H. Lempertz' Söhne, 14-11-1892",Heberle H. Lempertz' Söhne <Colonia>,NaN,NaN,Colonia,18921114,18921115,NaN,Sec. XIV/ XV,DIPINTI,COLLEZIONE ADOLF SCHUSTER
4,14355,87432,CA 18 1892 0322,Germania,Katalog einer Collection werthvoller Gemälde i...,"Asta Rudolph Lepke's Kunst-Auctions-Haus, 22-0...",Rudolph Lepke's Kunst-Auctions-Haus <Berlino>,NaN,NaN,Berlino,18920322,18920323,NaN,Sec. XIV/ XVI,DIPINTI; REPERTI ARCHEOLOGICI; MOBILI; MONETE,COLLEZIONE CARLO MORBIO


## Prepare graph and auxiliary methods

In [53]:
!pip install rdflib
import rdflib
from rdflib import Namespace, URIRef, Literal, Graph, ConjunctiveGraph, RDF, RDFS, XSD
from rdflib.store import Store

# Common namespaces
RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")
DC = Namespace("http://purl.org/dc/elements/1.1/")
CRM = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
LA = Namespace("https://linked.art/ns/terms/")
AAT = Namespace("http://vocab.getty.edu/aat/")

# Custom namespace
ZAC = Namespace("http://w3id.org/zac/")

# Create a context-aware graph using a Memory store
g = Graph(identifier="http://w3id.org/zac/catalogues")

# Bind namespaces to the graph
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)
g.bind("dc", DC)
g.bind("zac", ZAC)
g.bind("crm", CRM)
g.bind("la", LA)
g.bind("aat", AAT)

In [32]:
import re
import pandas as pd

def create_uri_string(input_string):
    """
    Creates a URI-friendly string from an input string by replacing spaces
    with underscores and substituting special characters with similar
    non-special characters, and converting the result to lowercase.

    Args:
        input_string: The string to convert.

    Returns:
        A URI-friendly string in lowercase, or None if the input is None or NaN.
    """
    if pd.isna(input_string):
        return input_string.tostring()

    # Replace spaces with underscores
    uri_string = input_string.strip().replace(" ", "_")

    # Define a mapping for special characters to similar non-special characters
    char_replacements = {
        'à': 'a', 'è': 'e', 'é': 'e', 'ì': 'i', 'ò': 'o', 'ù': 'u',
        'À': 'A', 'È': 'E', 'É': 'E', 'Ì': 'I', 'Ò': 'O', 'Ù': 'U',
        "'": "", '"': "", "‘": "", "’": "",  # Remove quotes
        "(": "", ")": "", "[": "", "]": "", "{": "", "}": "", # Remove brackets
        ",": "", ";": "", ":": "", ".": "", "!": "", "?": "", # Remove punctuation
        "&": "and",  # Replace ampersand
        "/": "_", # Replace slash
        "\\": "_", # Replace backslash
    }

    # Apply character replacements
    for old_char, new_char in char_replacements.items():
        uri_string = uri_string.replace(old_char, new_char)

    # Remove any remaining characters that are not alphanumeric, underscores, or hyphens
    uri_string = re.sub(r'[^\w-]', '', uri_string)

    # Convert the entire string to lowercase
    uri_string = uri_string.lower()

    return uri_string


In [33]:
!pip install langdetect
from langdetect import detect, DetectorFactory

# Set seed for reproducibility (optional)
DetectorFactory.seed = 0

def detect_language_iso(text):
    """
    Detects the language of a given text and returns its ISO 639-1 code.

    Args:
        text: The input string.

    Returns:
        The ISO 639-1 language code as a string, or None if language detection fails.
    """
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return None
    try:
        return detect(text)
    except:
        return None

In [34]:
import re

def extract_name_and_place(input_string):
    """
    Extracts a name and an optional place name from a string.

    Assumes the format is "Name <Place>" where <Place> is optional.

    Args:
        input_string: The input string.

    Returns:
        A tuple containing the name and the place name. The place name
        will be None if not present in the input string.
    """
    if pd.isna(input_string):
        return None, None

    # Regex to capture the name and the optional part in angle brackets
    match = re.match(r"([^<]+)(?:\s*<([^>]+)>)?", input_string.strip())

    if match:
        name = match.group(1).strip()
        place = match.group(2).strip() if match.group(2) else None
        return name, place
    else:
        # Return the original string as name if no match, and None for place
        return input_string.strip(), None

## Catalogues and Auctions

RDF triples generation

## Dictionaries

In [35]:
# LANGUAGES
langs = {"ger":"de", "fre":"fr","ita":"it", "eng":"en", "abs": "it"}
roles = {'commissaire-priseur': '300025208', # auctioneer
 'perito antiquario': '300025827', # antiquarian
 "casa d'aste": '300417515', # auction haouse
 "mercante d'arte": '300386253', # art dealer
 "storico dell'arte?": '300025541', # art historian
 "esperto": "300025829" # expert
         }

timespan_objects = {
 'Sec. XV/ XVI': ['300404465','300404510'],
 'Sec. XV/ XVIII': ['300404465','300404510','300404511','300404512'],
 'Sec. XIV/ XV': ['300404506','300404465'],
 'Sec. XIV/ XVI': ['300404506','300404465','300404510'],
 'Sec. XV/ XIX': ['300404465','300404465','300404510','300404510','300404512','300404513'],
 'Sec. XVI/ XVIII': ['300404510','300404511','300404512'],
 'Sec. XVI/ XIX': ['300404510','300404511','300404512','300404513'],
 'Sec. XVIII': ['300404512'],
 'Sec. XVII/ XVIII': ['300404511','300404512'],
 'Sec. XIV/ XVIII': ['300404506','300404465','300404510','300404511','300404512'],
 'Sec. XVI/ XVII': ['300404510','300404511'],
 'Sec. XVI/ XX': ['300404510','300404511','300404512','300404513','300404514'],
 'Sec. XV/ XVII': ['300404465','300404510','300404511'],
 'Sec. XVI/ IX': ['300404510','300404511','300404512','300404513'],
 'Sec. XIX': ['300404513'],
 'Sec. XIV/ XIX': ['300404506','300404465','300404510','300404511','300404512','300404513'],
 'Sec. XVII/ XIX': ['300404511','300404512','300404513'],
 'Sec. XVI': ['300404510'],
 ' Sec. XVII/ XVIII': ['300404511','300404512'],
 'Sec. XIV/ XVII': ['300404506','300404465','300404510','300404511'],
 'SeC. XVI/ XVIII': ['300404510', '300404511', '300404512'],
 'Sec. XIX/ XX': ['300404513','300404514'],
 'Sec. XIII/ XVII': ['300404505','300404506','300404465','300404510','300404511'],
 'Sec. XVII': ['300404511'],
 'Sec. XVI/ XIX ': ['300404510','300404511','300404512','300404513'],
 'Sec. XVIII/ XIX': ['300404512','300404513'],
 ' Sec. XV/ XVII': ['300404465','300404510','300404511'],
 'Sec. XIII/ XVIII': ['300404505','300404506','300404465','300404510','300404511','300404512'],
 'Sec. XII/ XIX': ['300404504','300404505','300404506','300404465','300404510','300404511','300404512','300404513'],
 'Sec. X/ XVI': ['300404502','300404503', '300404504','300404505','300404506','300404465','300404510'],
 'Sec. XVI / XIX': ['300404510','300404511','300404512','300404513'],
 'Sec. XV/ XX': ['300404465','300404514'],
 'SeC. XIX': ['300404513'],
 'Sec. XV/ XVIIII': ['300404465','300404510','300404511','300404512'],
 'Sec. IV': ['300404496'],
 'Sec. XIII/ XIX': ['300404505','300404506','300404465','300404510','300404511','300404512','300404513'],
 'Sec. XII/ XIII': ['300404504','300404505']}

# TODO AAT alignment
object_types = {'DIPINTI': '',
 "OGGETTI D'ARTE": '',
 'SCULTURE': '',
 'DISEGNI': '',
 'LIBRI': '',
 'TESSUTI': '',
 'MOBILI': '',
 'MEDAGLIE': '',
 'VETRI': '',
 'ARMI': '',
 'STAMPE': '',
 'CERAMICHE': '',
 'MINIATURE': '',
 'ARAZZI': '',
 'MAIOLICHE': '',
 'STRUMENTI MUSICALI': '',
 'DIPNTI': '',
 'SMALTI': '',
 'AVORII': '',
 'REPERTI ARCHEOLOGICI': '',
 'MONETE': '',
 'ARGENTI': '',
 'GIOIELLI': '',
 'ABITI': '',
 'STOFFE': '',
 'OROLOGI': '',
 'ARCHEOLOGIA': '',
 '': '',
 "GGETTI D'ARTE": '',
 'ARGENTERIE': '',
 'DIPINTI Sec. 19.': '',
 'MANOSCRITTI': '',
 'LENOBEL': '',
 'SCUTURE': '',
 'STRUMENTI SCIENTIFICI': '',
 'DIESEGNI': '',
 'DIPINTI Sec. 18.': '',
 'CERARMICHE': '',
 'LETTERE': '',
 'FERRO BATTUTO': '',
 'VINI': '',
 'DIPINIT': '',
 'INCISIONI': '',
 'TAPPETI': '',
 'AVORI': '',
 'DISRGNI': '',
 'VETRI ISTORIATI': '',
 'PITTURA': '',
 "0GGETTI D'ARTE": '',
 'QUADRI': '',
 'VASI': '',
 'PORCELLANE': '',
 'MAIOLICHE DI DELFT': '',
 'DIPINTI Sec. 19.-20.': '',
 'DISEGNI Sec. 19.-20.': '',
 'DIPINTI Sec. 18.-20.': '',
 'DISEGNI Sec. 18.-20.': '',
 'DIPINTI Sec. 17.-18.': '',
 'DISEGNI Sec. 16.-19.': '',
 'DIPINTI Sec. 15.-16.': '',
 'SCULTURE Sec. 13.-16.': '',
 'PISANELLO': '',
 'MINIATURE Sec. 13.-15.': '',
 'BLEIBINHAUS A.': '',
 "OGGETTI D'ARTE Sec. 18.-20.": '',
 'DIPINTI Sec. 14.-17.': '',
 'DIPINTI Sec. 16.-19.': '',
 'ARMATURE': '',
 'BRONZI Sec. 14.-18.': '',
 'SCHNEIDER & HANAU': '',
 'PITTURA Sec. 19.-20.': '',
 'LIPHART, Karl Eduard Freiherr von': '',
 'SCHEFIK PASCHA': '',
 'HORST, WILTH': '',
 "OGGETTI D'ARTE Sec. 14-16.": '',
 'ERGAS, RUDOLF': '',
 "OGGETTI D'ARTE Sec. 14.-16.": '',
 'SCHLÖSSER, KARL': '',
 'WOLFF, AUGUST': '',
 "OGGETTI D'ARTE Sec. 16.-18.": '',
 'JACOB DOPPLER': '',
 'DIPINTI Sec. 15.-19.': '',
 'SCULTURA Sec. 19.-20.': '',
 'DIPINTI NAPOLETANI': '',
 'ISENBURG, KARL VON': '',
 'ROTHSCHILD, D.': '',
 'GOEDECKER, CARL': '',
 'KÖSTER': '',
 'SULZBACH, EMIL': '',
 'DIPINTI Sec. 16.-18.': '',
 'DIPINTI Sec. 16.-20.': '',
 'CORINTH, LOVIS': '',
 'PITTURA Sec. 16.-19.': '',
 'BAYERN, GISELA von': '',
 'ACQUERELLI Sec. 19.-20.': '',
 'DIPINTI Sec. 20.': '',
 'DISEGNI Sec. 20.': '',
 'ARREDAMENTO Sec. 20.': '',
 'LÖWITH, WILHELM': '',
 "OGGETTI D'ARTE Sec. 15.-18.": '',
 'SPARR': '',
 'NEMES, MARCEL von': '',
 'DEYM': '',
 'HOHENTHAL': '',
 'LIPPERHEIDE, ELISABETH': '',
 'MAIOLICHE ITALIANE Sec. 15.-16.': '',
 'STRAUSS, OTTMAR': '',
 'ACQUARELLI Sec. 19.-20.': '',
 'PITTURA Sec. 19.-20': '',
 'ZIETHEN, FELIX': '',
 'PITTURA Sec. 16.-18.': '',
 'BOLIN': '',
 'LANDAU': '',
 'HEILAND': '',
 'PORCELLANE DI DOCCIA Sec. 18.': '',
 'EISENMANN': '',
 'SCHÜSSLER': '',
 'SCHWARZ': '',
 'Altkunst Antiquitäten': '',
 'Galeria van Diemen & Co': '',
 "OPERE D'ARTE": '',
 "COLLEZIOBNE D'HEUCQUEVILLE": '',
 'FRANCISCO GOYA': '',
 'BUDGE, EMMA': '',
 'ARREDAMENTO': '',
 'SCULTURE IN LEGNO Sec. 14.-18.': ''}

In [ ]:
import re

extracted_values_dict = {}

for index, row in df_asta.iterrows():
    banditore_value = row["BANDITORE"]

    if pd.notna(banditore_value):
        banditore_list = [item.strip() for item in banditore_value.split(';')]

        for item in banditore_list:
            match = re.search(r'<([^>]+)>', item)
            if match:
                extracted_value = match.group(1).strip()
                extracted_values_dict[extracted_value] = ""

#print("Extracted values dictionary:")
#display(extracted_values_dict)

type_dict = {}
for index, row in df_asta.iterrows():
    type_value = row["TIPI_OGGETTI_VENDUTI"]
    if pd.notna(type_value):
        # Split the string by semicolon if it exists
        type_list = [item.strip() for item in type_value.split(';')]
        for type_item in type_list:
          type_dict[type_item] = ""

#type_dict


In [ ]:
import pandas as pd

# Get unique values from the "CRONOLOGIA_OGG_VENDUTI" column, dropping NaN values
unique_cronologia_values = df_asta['CRONOLOGIA_OGG_VENDUTI'].dropna().unique()

# Create a dictionary with unique values as keys and empty strings as values
cronologia_dict = {value: "" for value in unique_cronologia_values}

print("Dictionary of unique 'CRONOLOGIA_OGG_VENDUTI' values:")
display(cronologia_dict)

Dictionary of unique 'CRONOLOGIA_OGG_VENDUTI' values:


{'Sec. XV/ XVI': '',
 'Sec. XV/ XVIII': '',
 'Sec. XIV/ XV': '',
 'Sec. XIV/ XVI': '',
 'Sec. XV/ XIX': '',
 'Sec. XVI/ XVIII': '',
 'Sec. XVI/ XIX': '',
 'Sec. XVIII': '',
 'Sec. XVII/ XVIII': '',
 'Sec. XIV/ XVIII': '',
 'Sec. XVI/ XVII': '',
 'Sec. XVI/ XX': '',
 'Sec. XV/ XVII': '',
 'Sec. XVI/ IX': '',
 'Sec. XIX': '',
 'Sec. XIV/ XIX': '',
 'Sec. XVII/ XIX': '',
 'Sec. XVI': '',
 ' Sec. XVII/ XVIII': '',
 'Sec. XIV/ XVII': '',
 'SeC. XVI/ XVIII': '',
 'Sec. XIX/ XX': '',
 'Sec. XIII/ XVII': '',
 'Sec. XVII': '',
 'Sec. XVI/ XIX ': '',
 'Sec. XVIII/ XIX': '',
 ' Sec. XV/ XVII': '',
 'Sec. XIII/ XVIII': '',
 'Sec. XII/ XIX': '',
 'Sec. X/ XVI': '',
 'Sec. XVI / XIX': '',
 'Sec. XV/ XX': '',
 'SeC. XIX': '',
 'Sec. XV/ XVIIII': '',
 'Sec. IV': '',
 'Sec. XIII/ XIX': '',
 'Sec. XII/ XIII': ''}

In [54]:
from logging import PlaceHolder
def role_assignment(uri_catalogue_creation, string_agent, uri_role):
  g.add((URIRef(uri_catalogue_creation+'_assignment_'+create_uri_string(string_agent)), RDF.type, CRM.E13_Attribute_Assignment))
  g.add((URIRef(uri_catalogue_creation+'_assignment_'+create_uri_string(string_agent)), CRM.P140_assigned_attribute_to, URIRef(uri_catalogue_creation) ))
  g.add((URIRef(uri_catalogue_creation+'_assignment_'+create_uri_string(string_agent)), CRM.P141_assigned, URIRef(ZAC[create_uri_string(string_agent)] ) ))
  g.add((URIRef(uri_catalogue_creation+'_assignment_'+create_uri_string(string_agent)), CRM.P177_assigned_property_type, CRM.P14_carried_out_by ))
  g.add((URIRef(uri_catalogue_creation+'_assignment_'+create_uri_string(string_agent)), CRM.P2_has_type, URIRef(uri_role) ))

import datetime


for index, row in df.iterrows():
    # CATALOGUE - Separate multiple catalogues
    id_cat = row['Nome cartella']
    id_cats = [create_uri_string(id.strip()) if ';' in id_cat else create_uri_string(id_cat) for id in id_cat.split(';')]
    title = row['TITOLO']
    secondary_title = row['ALTRITITOLI']
    date_cat = str(int(row['DATA PUBBLICAZIONE'])).strip()
    lang_cat = langs[row['LINGUE']]
    auction_name = row["Vendita all'asta"]
    author_person = row['AUTORI_PRINCIPALI'] # TODO reconciliation person / place
    author_org = row['ENTE AUTORE PRINCIPALE'] # TODO reconciliation org / place
    secondary_author_org = row["ENTE AUTORE SECONDARIO"]
    secondary_author_person = row["AUTORI_SECONDARI no banditori, mantenere solo curatori, esperti ecc."]
    for id in id_cats:
      g.add((URIRef(ZAC[id]), RDF.type, CRM.E31_Document))
      g.add((URIRef(ZAC[id]), CRM.P48_has_preferred_identifier, Literal(id)))
      # TODO add collocazione
      g.add((URIRef(ZAC[id]), CRM.P2_has_type, AAT["300026068"]))
      g.add((URIRef(ZAC[id]), RDFS.label, Literal(title, lang=lang_cat)))
      g.add((URIRef(ZAC[id]), CRM.P102_has_title, URIRef(ZAC[id+'_title']) ))
      g.add((URIRef(ZAC[id]), CRM.P72_has_language, URIRef(ZAC['lang_'+lang_cat]) ))
      g.add((URIRef(ZAC[id+'_lang_'+lang_cat]), RDF.type, CRM.E56_Language ))
      g.add((URIRef(ZAC[id+'_lang_'+lang_cat]), RDFS.label, Literal(lang_cat) ))
      g.add((URIRef(ZAC[id+'_title']), RDFS.label, Literal(title, lang=lang_cat)))
      g.add((URIRef(ZAC[id+'_title']), CRM.P2_has_type, URIRef(ZAC["primary_title"])))

      if pd.notna(secondary_title):
        g.add((URIRef(ZAC[id]), CRM.P102_has_title, URIRef(ZAC[id+'_secondary_title']) ))
        g.add((URIRef(ZAC[id+'_secondary_title']), RDFS.label, Literal(secondary_title, lang=lang_cat)))
        g.add((URIRef(ZAC[id+'_secondary_title']), CRM.P2_has_type, URIRef(ZAC["secondary_title"])))
      # CREATION
      g.add((URIRef(ZAC[id]), CRM.P94i_was_created_by, URIRef(ZAC[id+'_creation']) ))
      g.add((URIRef(ZAC[id+'_creation']), CRM.P82_at_some_time_within, Literal(date_cat, datatype=XSD.gYear)))
      g.add((URIRef(ZAC[id]), CRM.P70_documents, URIRef(ZAC[id+'_auction']) ))
      g.add((URIRef(ZAC[id+'_auction']), RDFS.label, Literal(auction_name) ))
      # AUTHORS
      if pd.notna(author_person):
        person, place = extract_name_and_place(author_person)
        g.add((URIRef(ZAC[id]), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(person)]) ))
        g.add((URIRef(ZAC[create_uri_string(person)]), RDFS.label, Literal(person) ))
        g.add((URIRef(ZAC[create_uri_string(person)]), RDF.type, CRM.E21_Person ))
        role_assignment(ZAC[id+'_creation'], person, AAT["300025492"])
        if place:
          g.add((URIRef(ZAC[create_uri_string(person)]), CRM.P74_has_current_or_former_residence, URIRef(ZAC[create_uri_string(place)]) ))
      if pd.notna(author_org):
        org, place = extract_name_and_place(author_org)
        g.add((URIRef(ZAC[id]), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(org)]) ))
        g.add((URIRef(ZAC[create_uri_string(org)]), RDFS.label, Literal(org) ))
        g.add((URIRef(ZAC[create_uri_string(org)]), RDF.type, CRM.E74_Group ))
        role_assignment(ZAC[id+'_creation'], org, AAT["300025492"])
        if place:
          g.add((URIRef(ZAC[create_uri_string(org)]), CRM.P74_has_current_or_former_residence, URIRef(ZAC[create_uri_string(place)]) ))

      if pd.notna(secondary_author_org):
          sec_org, place = extract_name_and_place(secondary_author_org)
          g.add((URIRef(ZAC[id]), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(sec_org)]) ))
          g.add((URIRef(ZAC[create_uri_string(sec_org)]), RDFS.label, Literal(sec_org) ))
          g.add((URIRef(ZAC[create_uri_string(sec_org)]), RDF.type, CRM.E74_Group ))
          role_assignment(ZAC[id+'_creation'], sec_org, ZAC["secondary_author"])
          if place:
            g.add((URIRef(ZAC[create_uri_string(sec_org)]), CRM.P74_has_current_or_former_residence, URIRef(ZAC[create_uri_string(place)]) ))

      if pd.notna(secondary_author_person):
          sec_person, place = extract_name_and_place(secondary_author_person)
          g.add((URIRef(ZAC[id]), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(sec_person)]) ))
          g.add((URIRef(ZAC[create_uri_string(sec_person)]), RDFS.label, Literal(sec_person) ))
          g.add((URIRef(ZAC[create_uri_string(sec_person)]), RDF.type, CRM.E21_Person ))
          role_assignment(ZAC[id+'_creation'], sec_person, ZAC["secondary_author"])
          if place:
            g.add((URIRef(ZAC[create_uri_string(sec_person)]), CRM.P74_has_current_or_former_residence, URIRef(ZAC[create_uri_string(place)]) ))


for index, row in df_asta.iterrows():
    # AUCTION - Separate multiple catalogues
    id_cat = row['INVENTARIO']
    id_cats = [create_uri_string(id.strip()) if isinstance(id_cat, str) and ';' in id_cat else create_uri_string(str(id_cat)) for id in str(id_cat).split(';')]
    auction_place = row["LUOGODIVENDITA"] # TODO reconciliation
    auction_date_start = row["DATA INIZIO ASTA"]
    auction_date_end = row["DATA FINE ASTA"]
    auction_date_label = ""
    xsd_auction_date_start, xsd_auction_date_end = None, None
    collections = row["COLLEZIONISTI_CONCATENATI"]
    collections_list = [id.strip() for id in str(collections).split(';')] if pd.notna(collections) else []
    organiser = row["AUTORI_PRINCIPALI/ENTE AUTORE"] # TODO reconciliation
    secondary_organiser = row["AUTORI_SECONDARI/ENTE AUTORE SECONDARIO"] # TODO reconciliation
    battitore= row["BANDITORE"] # TODO reconciliation
    battitori = [id.strip() for id in str(battitore).split(';')] if pd.notna(battitore) else []
    object_types_value = row["TIPI_OGGETTI_VENDUTI"]
    object_period = row["CRONOLOGIA_OGG_VENDUTI"]
    if pd.notna(auction_date_start):
      if str(auction_date_start).isdigit():
        #start = str(datetime.datetime.strptime(str(int(auction_date_start)), '%Y%m%d'))
        year= str(auction_date_start)[0:4]
        month= str(auction_date_start)[4:6]
        day= str(auction_date_start)[6:8]
        auction_date_label += str(year)+'/'+str(month)+'/'+str(day)
        xsd_auction_date_start = Literal(str(year)+"-"+str(month)+"-"+str(day), datatype=XSD.date)
      else:
        auction_date_label += str(auction_date_start).strip()
    if pd.notna(auction_date_end):
      if str(auction_date_end).isdigit():
        #start = str(datetime.datetime.strptime(str(int(auction_date_start)), '%Y%m%d'))
        end_year= str(auction_date_end)[0:4]
        end_month= str(auction_date_end)[4:6]
        end_day= str(auction_date_end)[6:8]
        auction_date_label += ' - ' + str(end_year)+'/'+str(end_month)+'/'+str(end_day)
        xsd_auction_date_end = Literal(str(end_year)+"-"+str(end_month)+"-"+str(end_day), datatype=XSD.date)
      else:
        if "idem" in str(auction_date_end):
            xsd_auction_date_end = xsd_auction_date_start
        else:
          auction_date_label += ' - ' + str(auction_date_end).strip()

    for id in id_cats:
      g.add(( URIRef(ZAC[id+'_auction']), RDF.type, CRM.E7_Activity )) # title and relation to catalogue already represented
      g.add(( URIRef(ZAC[id+'_auction']), CRM.P2_has_type, AAT["300054751"] ))
      # PLACE
      if pd.notna(auction_place):
        g.add(( URIRef(ZAC[id+'_auction']), CRM.P7_took_place_at, URIRef(ZAC[create_uri_string(auction_place)]) ))
        g.add(( URIRef(ZAC[create_uri_string(auction_place)]), RDFS.label, Literal(auction_place) ))
        g.add(( URIRef(ZAC[create_uri_string(auction_place)]), RDF.type, CRM.E53_Place ))
      # DATE
      auction_date_uri_string = create_uri_string(auction_date_label.strip().replace(' - ', '-'))
      g.add(( URIRef(ZAC[id+'_auction']), CRM["P4_has_time-span"], URIRef(ZAC[auction_date_uri_string]) ))
      g.add(( URIRef(ZAC[auction_date_uri_string]), RDF.type, CRM["E52_Time-Span"] ))
      g.add(( URIRef(ZAC[auction_date_uri_string]), RDFS.label, Literal(auction_date_label) ))
      if xsd_auction_date_start:
        g.add(( URIRef(ZAC[auction_date_uri_string]), CRM.P82a_begin_of_the_begin, xsd_auction_date_start ))
      if xsd_auction_date_end:
        g.add(( URIRef(ZAC[auction_date_uri_string]), CRM.P82b_end_of_the_end, xsd_auction_date_end ))
      # COLLECTIONS
      g.add(( URIRef(ZAC[id+'_auction']), CRM.P16_used_specific_object, URIRef(ZAC[id]) ))
      if collections_list: # Check if the list is not empty
        for collection in collections_list:
          if collection: # Check if the collection string is not empty
            g.add(( URIRef(ZAC[id+'_auction']), CRM.P16_used_specific_object, URIRef(ZAC[create_uri_string(collection)]) ))
            g.add(( URIRef(ZAC[create_uri_string(collection)]), RDFS.label, Literal(collection) ))
            g.add(( URIRef(ZAC[create_uri_string(collection)]), RDF.type, CRM.E78_Curated_Holding ))


      # ORGANISATION
      if pd.notna(organiser):
        organiser_name, place = extract_name_and_place(organiser)
        g.add(( URIRef(ZAC[id+'_auction']), CRM.P9_consists_of, URIRef(ZAC[id+'_organisation']) ))
        g.add(( URIRef(ZAC[id+'_organisation']), RDF.type, CRM.E7_Activity ))
        g.add(( URIRef(ZAC[id+'_organisation']), CRM.P2_has_type, URIRef(ZAC['auction_organisation']) ))
        g.add(( URIRef(ZAC[id+'_organisation']), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(organiser_name)]) ))
        g.add(( URIRef(ZAC[create_uri_string(organiser_name)]),RDFS.label, Literal(organiser_name) ))
        role_assignment(ZAC[id+'_organisation'], organiser_name, ZAC["main_organiser"])
        if place:
          g.add(( URIRef(ZAC[create_uri_string(organiser_name)]), CRM.P74_has_current_or_former_residence, URIRef(ZAC[create_uri_string(place)]) ))
      if pd.notna(secondary_organiser):
        sec_organiser_name, place = extract_name_and_place(secondary_organiser)
        g.add(( URIRef(ZAC[id+'_organisation']), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(sec_organiser_name)]) ))
        g.add(( URIRef(ZAC[create_uri_string(sec_organiser_name)]),RDFS.label, Literal(sec_organiser_name) ))
        role_assignment(ZAC[id+'_organisation'], sec_organiser_name, ZAC["secondary_organiser"])
      if battitori: # Check if the list is not empty
        for battitore_individual in battitori:
          if battitore_individual: # Check if the individual battitore string is not empty
            battitore_name, role = extract_name_and_place(battitore_individual)
            g.add(( URIRef(ZAC[id+'_auction']), CRM.P9_consists_of, URIRef(ZAC[id+'_auctioneer']) ))
            g.add(( URIRef(ZAC[id+'_auctioneer']), RDF.type, CRM.E7_Activity ))
            g.add(( URIRef(ZAC[id+'_auctioneer']), CRM.P2_has_type, URIRef(ZAC['auctioneering']) ))
            g.add(( URIRef(ZAC[id+'_auctioneer']), CRM.P14_carried_out_by, URIRef(ZAC[create_uri_string(battitore_name)]) ))
            g.add(( URIRef(ZAC[create_uri_string(battitore_name)]), RDFS.label, Literal(battitore_name) ))
            if role:
              aat_role = roles[role] if role in roles.keys() else '300025208' # auctioneer
              role_assignment(ZAC[id+'_auctioneer'], battitore_name, AAT[aat_role])

      # TYPE OF OBJECTS
      if pd.notna(object_types_value):
        object_types_list = [id.strip() for id in object_types_value.split(';')]
        for object_type in object_types_list:
          if object_type in object_types.keys():
            g.add(( URIRef(ZAC[id+'_auction']), CRM.P125_used_objects_of_type, URIRef(ZAC['object_type_'+create_uri_string(object_type)]) ))
            g.add(( URIRef(ZAC['object_type_'+create_uri_string(object_type)]), RDFS.label, Literal(object_type) ))
            g.add(( URIRef(ZAC['object_type_'+create_uri_string(object_type)]), CRM.P2_has_type, URIRef(ZAC['object_type']) ))
            if object_types[object_type] != '':
              g.add(( URIRef(ZAC['object_type_'+create_uri_string(object_type)]), RDFS.seeAlso, AAT[object_types[object_type]] ))
      # PERIOD OF OBJECTS
      # TODO and get year ranges, clean broader label
      if pd.notna(object_period):
        g.add(( URIRef(ZAC[id+'_auction']), CRM.P125_used_objects_of_type, URIRef(ZAC['object_period_'+create_uri_string(object_period)]) ))
        g.add(( URIRef(ZAC['object_period_'+create_uri_string(object_period)]), RDFS.label, Literal(object_period) ))
        g.add(( URIRef(ZAC['object_period_'+create_uri_string(object_period)]), CRM.P2_has_type, URIRef(ZAC['object_period']) ))
        if object_period in timespan_objects and timespan_objects[object_period]: # Check if the period is in the dictionary and has values
          for century in timespan_objects[object_period]: # TODO add labels
            g.add(( URIRef(ZAC['object_period_'+create_uri_string(object_period)]), RDFS.seeAlso, AAT[century] ))


g.serialize('zac_catalogues.trig', format='trig')

<Graph identifier=http://w3id.org/zac/catalogues (<class 'rdflib.graph.Graph'>)>